# Experiment 01

First Tests from the predictions.

In [1]:
import numpy as np
import pandas as pd
import math

import sys
sys.path.insert(1, 'C:\\Users\\Adrion\\Desktop\\TG\\DayTradeSystem\\Part1_Prediction')
import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')


def getPeriod2(df, begin, end, resetIndex = False):
    """
    Returns the df in the chosen interval
    
    Object begin: Start date forrmated as 'yyyy.mm.dd'.
    Object   end: End date 'yyyy.mm.dd'.

    returns a dataframe with the historic of the selected period
    """
    
    indexBegin = df[df['date']==begin].index[0]
    indexEnd = df[df['date']==end].tail(1).index[0]
    
    if (resetIndex):
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)].reset_index(drop=True)
    else: 
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)]

In [2]:
dfColumns = ['date','time','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']

TEST_BEGIN = '2020.06.01'
TEST_END = '2021.06.01'

In [3]:
df = pd.read_csv('../Data/PETR4_M15.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_M15.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

initialDf_PETR4 = getPeriod2(df, TEST_BEGIN, TEST_END)
initialDf_VALE3 = getPeriod2(df2, TEST_BEGIN, TEST_END)

In [4]:
initialDf_PETR4

,date,time,open,high,low,close
0,2020.06.01,10:00:00,19.50,19.59,19.38,19.59
1,2020.06.01,10:15:00,19.58,19.68,19.35,19.45
2,2020.06.01,10:30:00,19.46,19.67,19.35,19.64
3,2020.06.01,10:45:00,19.64,19.73,19.49,19.52
4,2020.06.01,11:00:00,19.52,19.74,19.48,19.68
...,...,...,...,...,...,...
7311,2021.06.01,15:45:00,27.14,27.14,27.05,27.06
7312,2021.06.01,16:00:00,27.06,27.09,27.01,27.02
7313,2021.06.01,16:15:00,27.01,27.17,27.01,27.14
7314,2021.06.01,16:30:00,27.15,27.31,27.11,27.27


In [5]:
initialDf_VALE3

,date,time,open,high,low,close
0,2020.06.01,10:00:00,49.43,49.56,49.14,49.45
1,2020.06.01,10:15:00,49.43,49.78,49.42,49.57
2,2020.06.01,10:30:00,49.58,49.79,49.29,49.72
3,2020.06.01,10:45:00,49.72,50.03,49.62,49.94
4,2020.06.01,11:00:00,49.94,49.97,49.70,49.86
...,...,...,...,...,...,...
7301,2021.06.01,15:45:00,113.84,113.95,113.61,113.72
7302,2021.06.01,16:00:00,113.72,114.00,113.70,113.99
7303,2021.06.01,16:15:00,113.99,114.00,113.90,113.96
7304,2021.06.01,16:30:00,113.97,114.00,113.88,113.88


In [6]:
pred_PETR4 = pd.read_pickle('../Data/PETR4_Pred_ytd.pkl')
pred_PETR4.head(5)

,high,low,high_pred,low_pred,date
600,19.89,19.35,19.46755,18.66315,2020.06.01
601,20.70,19.93,19.46890,18.66245,2020.06.02
602,21.20,20.71,19.83215,19.00570,2020.06.03
603,21.07,20.36,19.74140,18.89875,2020.06.04
604,22.28,21.34,19.93865,19.08815,2020.06.05


In [7]:
pred_VALE3 = pd.read_pickle('../Data/VALE3_Pred_ytd.pkl')
pred_VALE3.head(15)

,high,low,high_pred,low_pred,date
600,50.40,49.14,49.73290,48.67650,2020.06.01
601,50.35,49.60,50.51375,49.58665,2020.06.02
602,50.86,49.96,50.74020,49.89305,2020.06.03
603,52.13,50.09,51.01740,50.13460,2020.06.04
604,53.07,50.86,52.26270,51.18830,2020.06.05
605,52.08,50.52,52.10685,50.99675,2020.06.08
606,51.47,50.65,52.12025,50.84390,2020.06.09
607,51.66,50.31,52.24530,50.86215,2020.06.10
608,50.59,48.65,52.03815,50.71490,2020.06.12
609,50.94,48.79,50.89725,49.55625,2020.06.15


### Início do sistema

In [8]:
def getDay(df, day):
    return df[df['date']==day]

def buy(value, availableMoney, opr='normal'):
    lotValue = value*100
    quantity = math.floor(availableMoney/lotValue)
    
    amount = quantity*lotValue
    remainingMoney = availableMoney - amount
    
    return remainingMoney, quantity

def sell(value, amount, opr='normal'):
    if(opr == 'normal'):
        return value*amount*100
    else:
        return value*amount*100

In [9]:
def runStrategy(df, dailyDf, Money, stopLoss, std):
    size = len(df.index)

    Money = Money
    stopLoss = stopLoss

    orders = []

    for day in range(0,size):
        currentDay =  df.iloc[day].date

        lowPred = df.iloc[day].low_pred*(1+std)
        highPred = df.iloc[day].high_pred*(1-std)

        dayDf = getPeriod2(dailyDf, currentDay, currentDay)
        daySize = len(dayDf.index)

        op = 'none'
        quantity = 0
        trade = []

        for i in range(0,daySize):
            currentTime = dayDf.iloc[i]

            if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
                isStop = False
                if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
                aux = sell(currentTime.close, quantity)
                Money = Money + aux 
                trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
                isStop = False
                if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

                aux = sell(currentTime.close, quantity)
                Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
                trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif(op == 'none' and i < daySize-1 ):
                if(currentTime.close <= lowPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'buy'

                    trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]

                elif(currentTime.close >=  highPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'sell'

                    trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]
        print('eday '+str(day)+' money: '+str(Money))
    
    ordersDf = pd.DataFrame(orders)
    ordersDf.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
    return ordersDf

In [10]:
testPetr = runStrategy(pred_PETR4, initialDf_PETR4, 100000, 0.005, 0.01)
print('Accuracy: '+str(round(100*len(testPetr[testPetr['profit'] > 0].index)/len(testPetr.index), 2)))

testVale = runStrategy(pred_VALE3, initialDf_VALE3, 100000, 0.005, 0.01)
print('Accuracy: '+str(round(100*len(testVale[testVale['profit'] > 0].index)/len(testVale.index), 2)))

eday 0 money: 99369.0
eday 1 money: 96524.00000000003
eday 2 money: 98324.00000000003
eday 3 money: 97846.00000000001
eday 4 money: 100266.00000000003
eday 5 money: 100446.00000000003
eday 6 money: 100446.00000000003
eday 7 money: 104081.00000000004
eday 8 money: 104409.00000000003
eday 9 money: 102210.00000000004
eday 10 money: 102594.00000000004
eday 11 money: 106465.00000000007
eday 12 money: 108811.0000000001
eday 13 money: 112727.0000000001
eday 14 money: 110297.00000000009
eday 15 money: 110591.00000000009
eday 16 money: 108948.00000000013
eday 17 money: 111174.00000000013
eday 18 money: 110148.00000000013
eday 19 money: 109201.00000000013
eday 20 money: 111904.0000000001
eday 21 money: 112387.0000000001
eday 22 money: 112491.0000000001
eday 23 money: 112373.0000000001
eday 24 money: 112956.0000000001
eday 25 money: 113736.0000000001
eday 26 money: 111629.0000000001
eday 27 money: 114129.0000000001
eday 28 money: 116991.0000000001
eday 29 money: 116991.0000000001
eday 30 money: 1

In [11]:
testPetr.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
588,2021.05.27,62,13:15:00,Buy,25.88,16:45:00,Sell,25.91,0.03,186.0,False,161008.0
589,2021.05.28,60,10:45:00,Sell,26.64,11:00:00,Buy,26.82,-0.18,-1080.0,True,159928.0
590,2021.05.28,59,11:15:00,Sell,26.82,14:30:00,Buy,26.97,-0.15,-885.0,True,159043.0
591,2021.05.28,59,14:45:00,Sell,26.95,16:45:00,Buy,26.99,-0.04,-236.0,False,158807.0
592,2021.05.31,58,10:00:00,Sell,26.97,16:45:00,Buy,26.87,0.10,580.0,False,159387.0
593,2021.06.01,58,10:00:00,Buy,27.37,10:15:00,Sell,27.52,0.15,870.0,False,160257.0
594,2021.06.01,58,10:30:00,Buy,27.36,10:45:00,Sell,27.36,0.00,0.0,False,160257.0
595,2021.06.01,58,11:00:00,Sell,27.42,11:30:00,Buy,27.37,0.05,290.0,False,160547.0
596,2021.06.01,59,11:45:00,Buy,27.18,14:30:00,Sell,27.02,-0.16,-944.0,True,159603.0
597,2021.06.01,58,14:45:00,Buy,27.10,16:45:00,Sell,27.29,0.19,1102.0,False,160705.0


In [12]:
testVale.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
647,2021.05.28,10,10:00:00,Sell,111.15,10:15:00,Buy,110.26,0.89,890.0,False,120178.0
648,2021.05.28,10,11:00:00,Buy,110.27,12:30:00,Sell,111.08,0.81,810.0,False,120988.0
649,2021.05.28,10,12:45:00,Sell,111.17,14:00:00,Buy,111.75,-0.58,-580.0,True,120408.0
650,2021.05.28,10,14:15:00,Sell,112.13,14:30:00,Buy,112.70,-0.57,-570.0,True,119838.0
651,2021.05.28,10,14:45:00,Sell,112.29,16:45:00,Buy,111.59,0.70,700.0,False,120538.0
652,2021.05.31,10,10:00:00,Sell,113.62,10:15:00,Buy,114.48,-0.86,-860.0,True,119678.0
653,2021.05.31,10,10:30:00,Sell,114.06,11:00:00,Buy,114.75,-0.69,-690.0,True,118988.0
654,2021.05.31,10,11:15:00,Sell,114.63,16:45:00,Buy,114.78,-0.15,-150.0,False,118838.0
655,2021.06.01,10,10:00:00,Sell,117.54,14:30:00,Buy,114.19,3.35,3350.0,False,122188.0
656,2021.06.01,10,14:45:00,Buy,114.16,16:45:00,Sell,113.20,-0.96,-960.0,True,121228.0


In [13]:
df = pred_VALE3
dailyDf = initialDf_VALE3
size = len(df.index)

Money = 100000
stopLoss = 0.01
LastTrade = ['None',10,100]

orders = []

for day in range(0,size):
    currentDay =  df.iloc[day].date

    lowPred = df.iloc[day].low_pred*1.01
    highPred = df.iloc[day].high_pred*0.99
    
    dayDf = getPeriod2(dailyDf, currentDay, currentDay)
    daySize = len(dayDf.index)
    
    op = 'none'
    quantity = 0
    trade = []
    
    print('Begining day '+str(day)+' money: '+str(Money))
    for i in range(0,daySize):
        currentTime = dayDf.iloc[i]
        
        if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
            isStop = False
            if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
            aux = sell(currentTime.close, quantity)
            Money = Money + aux 
            trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
            orders.append(trade)
  
            quantity = 0
            op = 'none'
        
        elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
            isStop = False
            if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

            aux = sell(currentTime.close, quantity)
            Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
            trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
            orders.append(trade)
            
            quantity = 0
            op = 'none'
        
        elif(op == 'none' and i < daySize-1 ):
            if(currentTime.close <= lowPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'buy'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]
                
            elif(currentTime.close >=  highPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'sell'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
test2.tail(60)

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']

print('Accuracy: '+str(round(100*len(test2[test2['profit'] > 0].index)/len(test2.index), 2)))
test2.head(10)

Begining day 0 money: 100000
Begining day 1 money: 98621.0
Begining day 2 money: 99438.0
Begining day 3 money: 102215.99999999999
Begining day 4 money: 101360.99999999999
Begining day 5 money: 102124.99999999999
Begining day 6 money: 103022.99999999999
Begining day 7 money: 104223.0
Begining day 8 money: 103242.99999999999
Begining day 9 money: 103522.99999999997
Begining day 10 money: 107239.99999999999
Begining day 11 money: 107857.99999999997
Begining day 12 money: 106897.99999999999
Begining day 13 money: 106477.99999999997
Begining day 14 money: 107657.99999999997
Begining day 15 money: 107977.99999999996
Begining day 16 money: 107677.99999999996
Begining day 17 money: 106757.99999999996
Begining day 18 money: 107457.99999999997
Begining day 19 money: 108357.99999999999
Begining day 20 money: 110079.99999999999
Begining day 21 money: 110940.99999999999
Begining day 22 money: 110394.99999999997
Begining day 23 money: 108798.99999999999
Begining day 24 money: 109722.99999999996
Begi

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
0,2020.06.01,20,10:00:00,Sell,49.45,11:30:00,Buy,50.13,-0.68,-1360.0,True,98640.0
1,2020.06.01,19,11:45:00,Sell,49.99,16:45:00,Buy,50.00,-0.01,-19.0,False,98621.0
2,2020.06.02,19,10:00:00,Buy,49.80,10:30:00,Sell,50.05,0.25,475.0,False,99096.0
3,2020.06.02,19,10:45:00,Sell,50.15,11:30:00,Buy,50.07,0.08,152.0,False,99248.0
4,2020.06.02,19,11:45:00,Buy,50.06,13:00:00,Sell,50.04,-0.02,-38.0,False,99210.0
5,2020.06.02,19,13:15:00,Buy,49.92,15:00:00,Sell,50.01,0.09,171.0,False,99381.0
6,2020.06.02,19,15:15:00,Buy,49.97,16:45:00,Sell,50.00,0.03,57.0,False,99438.0
7,2020.06.03,19,10:00:00,Sell,50.75,10:30:00,Buy,50.31,0.44,836.0,False,100274.0
8,2020.06.03,20,10:45:00,Buy,50.09,11:00:00,Sell,50.53,0.44,880.0,False,101154.0
9,2020.06.03,19,11:15:00,Sell,50.69,11:45:00,Buy,50.31,0.38,722.0,False,101876.0
